In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Every Run

# Library Imports
import os
import librosa
import librosa.display
import numpy as np
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import IPython.display as ipd
from IPython.display import Audio

# Constants

SAMPLE_RATE = 22050*2
EMOTIONS = ["","Neutral","Calm","Happy","Sad","Angry","Fearful","Disgust","Surprised"]
EMOTIONS_GUESS = ["Neutral","Happy","Sad","Angry","Fearful","Disgust","Surprised"]

In [ ]:
# Load Data
data_Actors    = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_Actors.npy')
data_Labels    = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_Labels.npy')
data_mfccs     = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_mfccs.npy')
data_mfccsMA   = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_mfccsMA.npy')
data_mfccsMAWN = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_mfccsMAWN.npy')
data_mfccsWN   = np.load('/content/gdrive/MyDrive/FYP Code/Save Data/data_mfccsWN.npy')

for i in range(len(data_Labels)):
  temp = data_Labels[i] - 1
  if temp > 1: temp -= 1
  #if temp == 7: temp = 1
  data_Labels[i] = temp

In [ ]:
def dataMixer(data,mixer,ratio):
  output = data
  dataLength = len(data)
  target = int(ratio*dataLength)
  for i in range(dataLength):
    if random.randint(0,dataLength-i)<=target:
      output[i] = mixer[i]
      target -= 1
    
  return output

In [ ]:
data_mfccsMAr = dataMixer(data_mfccs,data_mfccsMA,0.5)
data_mfccsWNr = dataMixer(data_mfccs,data_mfccsWN,0.5)
data_mfccsWNrMAr = dataMixer(data_mfccsWNr,data_mfccsMA,1/3)

In [ ]:
#Validation Filter
filterLOO_valid = []
for i in range(len(data_Actors)):
  if data_Actors[i] == 23 or data_Actors[i] == 24:
    filterLOO_valid.append(True)
  else:
    filterLOO_valid.append(False)
filterLOO_valid = np.array(filterLOO_valid)


valid_X = data_mfccs[filterLOO_valid]
valid_XWN = data_mfccsWN[filterLOO_valid]
valid_XMA = data_mfccsMA[filterLOO_valid]
valid_X_Full = np.append(valid_X,np.append(valid_XWN,valid_XMA)).reshape([valid_X.shape[0]*3,13,216])

valid_X = np.expand_dims(valid_X,axis=3)
valid_XWN = np.expand_dims(valid_XWN,axis=3)
valid_XMA = np.expand_dims(valid_XMA,axis=3)
valid_X_Full = np.expand_dims(valid_X_Full,axis=3)

valid_y = data_Labels[filterLOO_valid]
valid_y_Full = np.append(valid_y,np.append(valid_y,valid_y))

In [ ]:
# Training Filter

filter_train = []
for i in range(len(data_Actors)):
  if data_Actors[i] == 23 or data_Actors[i] == 24 or data_Actors[i] > 24: # RAVDESS 2324
  #if data_Actors[i] == 23 or data_Actors[i] == 24: # RAVDESS + TESS 2324
  #if data_Actors[i] <= 24: # TESS 2324
  #if data_Actors[i] >=25: # RAVDESS 25
  #if data_Actors[i] ==25: # RAVDESS + TESS 25
  #if data_Actors[i] <=25: # TESS 25
    filter_train.append(False)
  else:
    filter_train.append(True)
filter_train = np.array(filter_train)


train_X = data_mfccsWNr[filter_train]
train_y = data_Labels[filter_train]

train_X, train_y = shuffle(train_X, train_y, random_state=42)
train_X = np.expand_dims(train_X,axis=3)

In [ ]:
print(np.sum(filter_train),np.sum(filterLOO_valid))
print(np.sum(filterLOO_valid | filter_train)) # Should be total number of data points in use - RAVDESS 24 * 52 = 1248; TESS 2 * 1400 = 2800;
print(np.sum(filterLOO_valid & filter_train)) # Should be 0

In [ ]:
class_weights = compute_class_weight('balanced', np.unique(train_y),train_y)

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LSTM, Lambda
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras import backend as K
from tensorflow.keras.optimizers import Adam

model = Sequential()
kernel = 5
model.add(Conv2D(32, 5,strides=2,padding='same',
                 input_shape=(13,216,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv2D(64, 5,strides=2,padding='same',))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, 5,strides=2,padding='same',))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,3)))
# model.add(Lambda(lambda x: K.squeeze(x, axis= 1)))
model.add(Flatten())

# model.add(LSTM(16))
# model.add(Dropout(0.5))

model.add(Dense(7))
model.add(Activation('softmax'))
# opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)


opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
modelNumber = "035d"
epochsComplete = 0

In [ ]:
numEpochs = 10

for rep in range(numEpochs):
  cnnhistory = model.fit(train_X, train_y, batch_size=16, initial_epoch=epochsComplete, epochs=epochsComplete+1, validation_data=(valid_X_Full,valid_y_Full))
  epochsComplete += 1
  if epochsComplete == 1:
    myHistory = cnnhistory.history.copy()
  else:
    for x in cnnhistory.history.keys():
      myHistory[x] = myHistory[x] + cnnhistory.history[x]
  if epochsComplete < 10:
    temp = "0"
  else:
    temp = ""
  model.save("gdrive/My Drive/FYP Code/Save Data/Models/model" + modelNumber + "_" + temp + str(epochsComplete) + "epoch.h5")
  print(modelNumber,temp + str(epochsComplete))

  model.evaluate(valid_X,valid_y)
  model.evaluate(valid_XWN,valid_y)
  model.evaluate(valid_XMA,valid_y)

In [ ]:
myHistory

In [ ]:
# Load Previous Model

modelNumber = "005"
epochsComplete = 30

if epochsComplete < 10:
  temp = "0"
else:
  temp = ""
pathStr = "gdrive/My Drive/FYP Code/Save Data/Models/model"+ modelNumber +"_"+ temp + str(epochsComplete) +"epoch.h5"
print(pathStr)
model = keras.models.load_model(pathStr)
model.evaluate(valid_X,valid_y)
model.evaluate(valid_XWN,valid_y)
model.evaluate(valid_XMA,valid_y)

In [ ]:
# Test on self-recorded files

testFile = 'gdrive/My Drive/FYP Code/Recording4.m4a'
X, sample_rate = librosa.load(testFile, res_type='kaiser_fast',duration=2.5,sr=SAMPLE_RATE,offset=0)
Audio(X,rate = sample_rate, autoplay = True)

In [ ]:
# Generate Guess

temp = np.zeros((1,13,216))
sample_rate = np.array(sample_rate)
mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)

result = np.zeros((13,216))
result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
temp[0] = result

t = np.expand_dims(temp,axis=3)

myArr = model.predict(t)

guess = np.argmax(myArr)

temp = ["Neutral","Surprised","Happy","Sad","Angry","Fearful","Disgust"]
print(temp[guess])

if guess == 0: guess + 1
else: guess = guess + 2

print(myArr)
print(guess)


print(EMOTIONS[guess])